## Algorytmy ewolucyjne

In [1]:
import numpy as np
import pandas as pd

Napisać podstawowy algorytm genetyczny z mutacją gaussowską i krzyżowaniem jednopunktowym. Sprawdzić działanie algorytmu na funkcji x2+y2+2z2 oraz na pięciowymiarowej funkcji Rastrigina.

In [2]:
def my_func(individual):
    x, y, z = individual
    return x**2 + y**2 + 2*z**2

In [63]:
np.apply_along_axis(my_func, 1, population)

array([ 8.38438842,  7.32375471,  2.74712871, 16.73659341,  5.90368977,
        4.76944414,  3.45944955,  2.67315835,  0.90421603,  1.41733162,
        0.2127339 ,  3.88729224,  8.50524247,  2.29544059,  6.27622929,
        1.56575907,  2.35242702,  0.90764445,  0.77827086,  0.85615449,
        1.93444612,  1.79311223,  1.60470967,  2.11745316, 14.44908079,
        0.40520237,  6.36132461,  4.56013743,  5.20891348,  4.66564227,
        1.01994977,  2.58339349,  1.75100626,  9.07654847,  5.97251812,
        0.82402216,  0.11034648,  1.72174618,  6.02874043,  2.06805091,
        1.94931963,  7.08509613, 13.35118367,  3.79449024,  2.96363757,
        1.02181331,  4.3151015 , 15.54580029, 10.79205068,  0.97234349,
        3.11013846,  2.85419148,  5.56834535,  6.3953319 ,  5.72280047,
        0.76120645,  0.49023696,  6.55449339,  0.86568494, 11.98851225,
        1.57709778,  1.35671655,  0.52498599,  1.38295247,  3.34082365,
        1.48042381,  2.09233647,  1.80686448,  1.73240453,  7.90

In [81]:
class AE:
    
    def __init__(self, dimension, population_size, generations=100):
        self.dimension = dimension
        self.population_size = population_size
        self.population = np.random.randn(population_size, dimension)
        self.generations = generations
        self.mutation_rate = None
        self.crossover_rate = None
        self.fitness_function = None
    
    def point_crossover(self, parent1, parent2):
        if np.random.rand() < self.crossover_rate:
            point = np.random.randint(1, self.dimension)
            child1 = np.concatenate((parent1[:point], parent2[point:]))
            child2 = np.concatenate((parent2[:point], parent1[point:]))
            return child1, child2
        return parent1, parent2
    
    def gaussian_mutation(self, individual):
        if np.random.rand() < self.mutation_rate:
            individual += np.random.normal(0, 1, self.dimension)
        return individual
    
    def evaluation(self, population):
        return np.apply_along_axis(self.fitness_function, 1, population)
    
    def wheel_selection(self, population):
        fitness = self.evaluation(self.population)
        probabilities = fitness / np.sum(fitness)
        selected_indices = np.random.choice(self.population_size, self.population_size, p=probabilities)
        return population[selected_indices]
    
    def choose_two_random_indexes(self, population_size):
        # Choose two distinct indexes
        idx1, idx2 = np.random.choice(population_size, 2, replace=False)
        return idx1, idx2
    
    def train(self, fitness_function, mutation_rate=0.1, crossover_rate=0.8):
        
        self.fitness_function = fitness_function
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        
        for generation in range(self.generations):
            
            fitness = self.evaluation(self.population)

            new_population = []

            while len(new_population) < self.population_size:
                idx1, idx2 = self.choose_two_random_indexes(self.population_size)
                child1, child2 = self.point_crossover(self.population[idx1], self.population[idx2])
                child1 = self.gaussian_mutation(child1)
                child2 = self.gaussian_mutation(child2)
                new_population.extend([child1, child2])

            population = np.array(new_population[:self.population_size])
            self.population = wheel_selection(population)

        
        fitness = np.array([fitness_function(ind) for ind in population])
        best_individual = population[np.argmin(fitness)]
        best_fitness = fitness_function(best_individual)

In [79]:
ae = AE(3, 50)

In [80]:
ae.train(my_func)

NameError: name 'dimension' is not defined